# Covid-19: From model prediction to model predictive control

## A demo of the deterministic modeling framework

*Original code by Ryan S. McGee. Modified by T.W. Alleman in consultation with the BIOMATH research unit headed by prof. Ingmar Nopens.*

MIT license 2020 by T.W. Alleman, BIOMATH, Ghent University. 

Our code implements a SEIRS infectious disease dynamics models with extensions to model the effect quarantining detected cases. Using the concept of 'classes' in Python 3, the code was integrated with our previous work and allows to quickly perform Monte Carlo simulations, calibrate model parameters and calculate an *optimal* government policies using a model predictive controller (MPC). A white paper and souce code of our previous work can be found on the Biomath website. 

https://biomath.ugent.be/covid-19-outbreak-modelling-and-control

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from ipywidgets import interact,fixed,FloatSlider,IntSlider,ToggleButtons
import pandas as pd
import datetime
import scipy
#import coronaHelper2 as cH --> will be phased out
from scipy.integrate import odeint
import matplotlib.dates as mdates
import matplotlib
import scipy.stats as st
import networkx
import models

## Introduction

### Model dynamics

In this work, we extended the SEIR model to model the disease spread with a higher resolution. To this end, the *infected pool* is split into four types of infectiousness: 1) supermild (SM): people who show little to no symptoms at all, 2) mild (M): people with noticable, mild covid-19 symptoms, 3) heavy (H): people hospitalised but not in need of intensive care and 4) critical (C): people hospitalised and in need of intensive care. It takes several days before a heavy or critical infection becomes so severe that these patients need to be hospitalised. To this end, the pool of heavily and critically infected patients is split in two: not yet hospitalised (H, C) and hospitalised (HH,CH). The *removed pool* from the classical SEIR model is explicitly split into an immune (I) and dead (F) pool. People from the susceptible, exposed, supermild infected, mild infected and immune pool can be quarantined after having tested positive for covid-19. In the flowchart below, quarantine pools are denoted by the *Q*-suffix. Note that for individuals in the susceptible and immune pools, this corresponds to receiving a false positive test.

<img src="../figs/flowchart2.jpg" alt="drawing" width="400"/>

### Deterministic vs. Stochastic framework

The extended SEIR model is implemented using two frameworks: a deterministic and a stochastic (network based) framework. **This Jupyter Notebooks is a demo of the deterministic model, the stochastic model works but is not yet integrated with our current methods.** A deterministic implementation of the extended SEIRS model captures important features of infectious disease dynamics, but it assumes uniform mixing of the population (i.e. every individual in the population is equally likely to interact with every other individual). The deterministic approach results in a set of N ordinary differential equations, one for every of the N ’population pools’ considered. The main advantage of the deterministic model is that a low amount of computational resources are required while still maintaining an acceptable accuracy. The deterministic framework allows to rapidly explore scenarios and perform optimisations which require thousands of function evaluations. 

However, it is often important to consider the structure of contact networks when studying disease transmission and the effect of interventions such as social distancing and contact tracing. The main drawback of the deterministic approach is the inability to simulate contact tracing, which is one of the most promising measures against the spread of sars-cov-2. For this reason, the SEIRS dynamics depicted in on the above flowchart can be simulated on a stochastic dynamical network. This advantages include a more detailed analysis of the relationship between social network structure and effective transmission rates, including the effect of network-based interventions such as social distancing, quarantining, and contact tracing. The first drawback is the increased amount of computational resources required, leading to the inability to perform optimisations on a personal computer. Instead, high performance computing infrastructure is needed. The second drawback is the need for more data and/or assumptions on social interactions and how government measures affect these social interactions.

#### Deterministic equations
The dynamics of the deterministic system are mathematically formulated as the rate of change of each population pool shown in the above flowchart. This results in the following system of ordinary differential equations,

\begin{eqnarray}
\dot{S} &=& - \beta \cdot N_c \Big( \frac{E+SM}{N} \Big) \cdot S  - \theta_{\text{S}} \psi_{\text{FP}} \cdot S + SQ/d_{\text{q,FP}} + \zeta \cdot R,\\
\dot{E} &=&  \beta \cdot N_c  \Big( \frac{E+SM}{N} \Big) \cdot S - B / \sigma - \theta_{\text{E}} \psi_{\text{PP}} \cdot E,\\
\dot{SM} &=& \text{sm}/\sigma  \cdot E - SM/d_{\text{sm}} - \theta_{\text{SM}} \psi_{\text{PP}} \cdot SM,\\ 
\dot{M} &=&  \text{m} / \sigma \cdot E - M/d_{\text{m}} - \theta_{\text{M}} \psi_{\text{PP}} \cdot M,\\
\dot{H} &=&  \text{h} / \sigma \cdot E + h / \sigma \cdot EQ - H/d_{\text{hospital}},\\
\dot{C} &=&  \text{c} / \sigma \cdot E + c / \sigma \cdot EQ - C/d_{\text{hospital}},\\
\dot{HH} &=& H/d_{\text{hospital}} - HH/d_{h}\\
\dot{CH} &=& C/d_{\text{hospital}} - m_c \cdot CH/d_{\text{cf}} - (1-m_c) \cdot CH/d_{\text{cr}}\\
\dot{F} &=& m_c \cdot CH/d_{\text{cf}}\\
\dot{R} &=& SM/d_{\text{sm}} + M/d_{\text{m}} + HH/d_{\text{h}} + (1-m_h) \cdot CH/d_{\text{cr}} + SMQ/d_{\text{sm}} \\ && + MQ/d_{\text{m}} + RQ/d_{\text{q,FP}} - \zeta \cdot R \\
\dot{SQ} &=& \theta_{\text{S}} \psi_{\text{FP}} \cdot S - SQ/d_{\text{q,FP}} \\
\dot{EQ} &=& \theta_{\text{E}} \psi_{\text{PP}} \cdot E - EQ/\sigma\\
\dot{SMQ} &=& \theta_{\text{SM}} \psi_{\text{PP}} \cdot SM + sm/\sigma \cdot EQ - SMQ/d_{\text{sm}} \\
\dot{MQ} &=& \theta_{\text{M}} \psi_{\text{PP}} \cdot M + m/\sigma \cdot EQ - MQ/d_{\text{m}}\\
\dot{RQ} &=& \theta_{\text{R}} \psi_{\text{FP}} \cdot R - RQ/d_{\text{q,FP}}
\end{eqnarray}

#### Stochastic equations
Consider a network graph G like the one depicted below, representing individuals (nodes) and their interactions (edges). Each individual (node) has a state (S, E, SM, M, H, C, HH, CH, R, F, SQ, SMQ, MQ or RQ). The set of nodes adjacent (connected by an edge) to an individual defines their set of "close contacts" (highlighted in black). At a given time, each individual makes contact with a random individual from their set of close contacts with probability (1-p) or with a random individual from anywhere in the network (highlighted in blue) with probability p. The latter global contacts represent individuals interacting with the population at large. These are individuals outside of ones inner social circle, such as on public transit, at an event or chatting to an old acquaintance etc. When a susceptible individual interacts with an exposed or infectious individual they become exposed themselves. The parameter p defines the locality of the network: for p=0 an individual only interacts with their close contacts, while p=1 represents a uniformly mixed population. **For a sufficiently large network G, and $p = 1$, the stochastic and deterministic framework will yield the same result**. Social distancing interventions ifluences both the locality and connectivity of the network. It is expected that social distancing lowers the average number of interactions an individual has each day (lower connectivity of the network). But as a result, the fraction of interactions in the inner circle increases, lowering p.

<img src="../figs/network_contacts.png"
     alt="network"
     height="180" width="200"     
     style="float: left; margin-right: 10000px;" /> 

Each node $i$ has a state $X_i$ that updates according to a probability transition rate. The dynamics of the stochastic system are mathematically formulated as probabilities of transitioning between the pools shown in the model flowchart. In addition, back tracking of confirmed cases can be included in the model,

\begin{eqnarray}
\text{Pr}(X_i = S \rightarrow E) &=&  \Big[ \underbrace{p \cdot \frac{\beta (E + SM)}{N}}_{\text{infection through random encounter}} \\ 
&& + \underbrace{(1-p) \cdot \frac{\beta \sum_{j \in C_G(i)} (\delta_{X_j = E}+\delta_{X_j = SM})}{C_G (i)}}_{\text{infection through inner circle}} \Big] \delta_{X_i=S},\\
\text{Pr}(X_i = E \rightarrow SM) &=&  (sm/\sigma) \cdot \delta_{X_i = E},\\
\text{Pr}(X_i = E \rightarrow M) &=&  (m/\sigma) \cdot \delta_{X_i = E},\\
\text{Pr}(X_i = E \rightarrow H) &=&  (h/\sigma) \cdot \delta_{X_i = E},\\
\text{Pr}(X_i = E \rightarrow C) &=&  (c/\sigma) \cdot \delta_{X_i = E},\\
\text{Pr}(X_i = H \rightarrow HH) &=&  (1/d_{\text{hospital}}) \delta_{X_i = H},\\
\text{Pr}(X_i = C \rightarrow CH) &=&  (1/d_{\text{hospital}}) \delta_{X_i = C},\\
\text{Pr}(X_i = SM \rightarrow R) &=& (1/d_{\text{sm}}) \cdot \delta_{X_i = SM},\\
\text{Pr}(X_i = M \rightarrow R) &=& (1/d_{\text{m}}) \cdot \delta_{X_i = M},\\
\text{Pr}(X_i = HH \rightarrow R) &=& (1/d_{\text{h}}) \cdot \delta_{X_i = HH},\\
\text{Pr}(X_i = CH \rightarrow R) &=& (1-m_c)/d_{\text{cr}} \cdot \delta_{X_i = HH},\\
\text{Pr}(X_i = CH \rightarrow F) &=& (m_c/d_{\text{cf}}) \cdot \delta_{X_i = HH},\\
\text{Pr}(X_i = S \rightarrow SQ) &=& \Big[\theta_S + \phi_S \big[ \sum_{j \in C_G(i)} (\delta_{X_j = SQ}+\delta_{X_j = EQ}+\delta_{X_j = SMQ} \\
&&+\delta_{X_j = MQ}+\delta_{X_j = RQ}+\delta_{X_j = HH}+\delta_{X_j = CH}) \big] \Big] \cdot \psi_{\text{FP}} \cdot \delta_{X_i = S}, \\
\text{Pr}(X_i = E \rightarrow EQ) &=& \Big[\theta_E + \phi_E \big[ \sum_{j \in C_G(i)} (\delta_{X_j = SQ}+\delta_{X_j = EQ}+\delta_{X_j = SMQ} \\
&&+\delta_{X_j = MQ}+\delta_{X_j = RQ}+\delta_{X_j = HH}+\delta_{X_j = CH}) \big] \Big] \cdot \psi_{\text{PP}} \cdot \delta_{X_i = E}, \\
\text{Pr}(X_i = SM \rightarrow SMQ) &=& \Big[\theta_{SM} + \phi_{SM} \big[ \sum_{j \in C_G(i)} (\delta_{X_j = SQ}+\delta_{X_j = EQ}+\delta_{X_j = SMQ} \\
&&+\delta_{X_j = MQ}+\delta_{X_j = RQ}+\delta_{X_j = HH}+\delta_{X_j = CH}) \big] \Big] \cdot \psi_{\text{PP}} \cdot \delta_{X_i = SM}, \\
\text{Pr}(X_i = M \rightarrow MQ) &=& \Big[\theta_{M} + \phi_{M} \big[ \sum_{j \in C_G(i)} (\delta_{X_j = SQ}+\delta_{X_j = EQ}+\delta_{X_j = SMQ} \\
&&+\delta_{X_j = MQ}+\delta_{X_j = RQ}+\delta_{X_j = HH}+\delta_{X_j = CH}) \big] \Big] \cdot \psi_{\text{PP}} \cdot \delta_{X_i = M} \\
\text{Pr}(X_i = R \rightarrow RQ) &=& \Big[\theta_{R} + \phi_{R} \big[ \sum_{j \in C_G(i)} (\delta_{X_j = SQ}+\delta_{X_j = EQ}+\delta_{X_j = SMQ} \\
&&+\delta_{X_j = MQ}+\delta_{X_j = RQ}+\delta_{X_j = HH}+\delta_{X_j = CH}) \big] \Big] \cdot \psi_{\text{FP}} \cdot \delta_{X_i = R}\\
\text{Pr}(X_i = SQ \rightarrow S) &=& (1/d_{q,FP}) \cdot \delta_{X_i = SQ},\\
\text{Pr}(X_i = EQ \rightarrow SMQ) &=& sm \cdot \sigma \cdot \delta_{X_i = EQ},\\
\text{Pr}(X_i = EQ \rightarrow MQ) &=& m \cdot \sigma \cdot \delta_{X_i = EQ},\\
\text{Pr}(X_i = EQ \rightarrow H) &=& h \cdot \sigma \cdot \delta_{X_i = EQ},\\
\text{Pr}(X_i = EQ \rightarrow C) &=& h \cdot \sigma \cdot \delta_{X_i = EQ},\\
\text{Pr}(X_i = SMQ \rightarrow R) &=& h (1/d_{sm}) \cdot \delta_{X_i = SMQ},\\
\text{Pr}(X_i = MQ \rightarrow R) &=& h (1/d_{m}) \cdot \delta_{X_i = MQ},\\
\text{Pr}(X_i = RQ \rightarrow R) &=& h (1/d_{\text{hospital}}) \cdot \delta_{X_i = RQ},\\
\text{Pr}(X_i = R \rightarrow S) &=& \zeta \cdot \delta_{X_i = R},\\
\end{eqnarray}

where $\delta_{X_i = A} = 1$ if the state of $X_i$ is A, or 0 if not, and where $C_G(i)$ denotes the set of close contacts of node i (adjacent nodes). For large populations and $p = 1$, this stochastic model approaches the same dynamics as the deterministic SEIRS model.

### Model parameters

In the above equations, S stands for susceptible, E for exposed, SM for supermild, M for mild, H for heavy, C for critical, HH for heavy and hospitalised, CH for critical and hospitalised, F for dead, R for immune. The quarantined states are SQ, which stands for susceptible and quarantined, EQ for exposed and quarantined, SMQ for supermild infected and quarantined, MQ for mildly infected and quarantined and RQ for recovered and quarantined. N stands for the total population. The clinical parameters are: sm, m, z, h: the chance of having an asymptotic, mild, heavy or critical infection. Based on reported cases in China and travel data, Li et al. (2020b) estimated that 86 % of coronavirus infections in the country were "undocumented" in the weeks before officials instituted stringent quarantines. A biotech company in Iceland that has tested more than 9,000 people found that around 50 percent of those who tested positive said they were asymptomatic. We assume that 50 % of all infected cases are supermild (asymptotic) and hence,

$$sm = 0.50$$

Based on previously reported estimates of the distribution between mild, severe and critical cases (Wu and McGoogan, 2020), it is then calculated that the chance of contrapting a mild, severe and critical infection is,

$$m = (1 − 0.50) · 0.81 = 0.405,$$
$$h = (1 − 0.50) · 0.14 = 0.070,$$
$$c = (1 − 0.50) · 0.05 = 0.025.$$

$d_{sm}$ , $d_m$ , $d_h$ : the number of symptomic days in case of a supermild, mild or heavy infection. $d_{hf}$ is the time from hospitalisation until death in case of a critical infection while $d_{hr}$ is the recovery time for critical cases. Zhou et al. (2020) performed a retrospective study on 191 Chinese hospital patients and determined that the time from illness onset to discharge or death was 22.0 days (18.0-25.0, IQR) and 18.5 days (15.0-22.0, IQR) for survivors and victims respectively. Using available preliminary data, the World Health Organisation estimated the median time from onset to clinical recovery for mild cases to be approximately 2 weeks and to be 3-6 weeks for patients with severe or critical disease (WHO, 2020). Based on this report, we assume a recovery time of three weeks for heavy infections. d hospital : the time before heavily or critically infected patients reach the hospital. On average this is 9.1 days (Li et al., 2020a). m c : the mortality in case of a critical infection, which is roughly 50% (Wu and McGoogan, 2020). It is assumed that if the number of critical cases surpasses the total number of ICU beds, the mortality is changed to the average between 0.49 and 1, with the fraction of patients in critical condition receiving care as a weight. This is formulated mathematically in the following way,

\begin{equation}
m_h = \underbrace{\Big( \frac{ICU_{\text{max}}}{H} \Big) \cdot 0.49}_{\text{critical patient receives care}} + \underbrace{\Big( \frac{H - ICU_{\text{max}}}{H} \Big) \cdot 1.00}_{\text{critical patient receives no care}}\ .
\end{equation}

$\sigma$: the incubation period in days. The incubation period is assumed to be Erlang distributed as reported by Li et al. (2020a). The average incubation time is 5.2 days. $\zeta$: can be used to model the effect of re-susceptibility and seasonality of a disease. Throughout this demo, we assume $\zeta = 0$ because data on seasonality is not yet available at the moment. We thus assume permanent immunity after recovering from the infection.

The transmission rate of the disease in the deterministic model depends on the product of three contributions. The first contribution, $(E+SM)/N$, is the probability of encountering a contagious individual. The second contribution, $N_c$, is the average number of human-to-human interactions per day. As previously explained, **in the deterministic framework, this means all $N_c$ contacts within the population are random**. In this work, we explictly split $\beta$ and $N_c$ because this has the following advantages: 1) $\beta$ is now a disease characteric, independent of social interactions. 2) The goal of this work is to demonstrate the concept of social control to contain the outbreak using model predictive control. By splitting $\beta$ and $N_c$, the controlled variable is the number of random contacts $N_c$ which is more comprehendible to the reader of this text. The number of human-human interactions per day is estimated using the Social Contact Rates (SOCRATES) Data Tool (Willem et al., 2020). The dataset on human-human interactions for Belgium is based on a 2008 study by Mossong, which kept track of 750 participants and their interactions (8878 interactions) in Belgium. The dataset includes both physical and non-physical interactions of any duration. The third contribution, $\beta$, is the probability of contracting sars-cov-2 when encoutering a contagious individual. The testing and quarantine parameters are: $\theta_{S}$, $\theta_{E}$, $\theta_{SM}$, $\theta_{M}$, $\theta_{R}$ expressed as the number of susceptibles, exposed, supermild and mild individuals tested each day. It is difficult to model a concrete testing scenario in the deterministic framework because no back tracing can be coupled to the tests. The deterministic model only considers random testing of the population. It uses the total daily testing capacity to calculate the amount of individuals in the S, E, SM, M and R pool that receive a test. $\psi_{PP}$: probability of correctly identifiying and quarantining an exposed or infected person. $\psi_{FP}$: probability of falsly identifying and quarantining a susceptible or recovered individual. Both parameters can be calculated using the reliability of the test and Bayes theorem. For now, we assume no false positives. $d_{\text{q,FP}}$: the duration of the quarantine in the case of a false positive, assumed to be 14 days.

### Social interaction data

#### Social Contact Rates (SOCRATES) Data Tool

https://lwillem.shinyapps.io/socrates_rshiny/

1. What is the average number of daily human-to-human contacts of the Belgian population? Include all ages, all genders and both physical and non-physical interactions of any duration. To include all ages, type: *0,60+* in the *Age Breaks* dialog box.
2. What is the average number of physical human-to-human contacts of the Belgian population? Include all ages, all genders and all durations of physical contact.
3. What is the average number of physical human-to-human contacts of at least 1 hour of the Belgian population?
4. Based on the above results, how would you estimate $N_c$ in the deterministic model?
5. Based on the above results, how would you estimate $p$ in the stochastic model? Recall that $p$ is the fraction of *random contacts* a person has on a daily basis, while $(1-p)$ is the fraction of *inner circle contacts* a person has on a daily basis.

#### Google COVID-19 Community Mobility Reports

https://www.google.com/covid19/mobility/

#### London School of Hygiene

https://www.thelancet.com/journals/lanpub/article/PIIS2468-2667(20)30073-6/fulltext

## Performing simulations

### Without age-structuring

#### The 'SEIRSAgeModel' class

The basic concept of object oriented programming in Python 3 is illustrated schematically below. An object, created by calling a class, should be seen as a 'box' containing 'tools'. First, the model parameters, the initial condition and the monte-carlo simulation settings are put inside. The toolbox 'SEIRSAgeModel' not only contains the necessary code to simulate the model, but also contains several convenient functions for data visualisation, calibration of model parameters and model predictive control. The advantage of using an object instead of nested functions is the fact that function arguments don't explicitly have to be passed to the helper functions every time these are called by the user. Rather, the parameters are stored in our 'SEIRSAgeModel' toolbox and can be used by the class functions at any time. This drastically enhances the readability of the code.

<img src="../figs/SEIRSAgeModel.jpg"
     alt="class"
     height="600" width="700"     
     style="float: left; margin-right: 500px;" /> 

As of now (18/04/2020), the SEIRSAgeModel contains 7 functions which can be grouped in two parts: 1) functions to run and visualise simulations and 2) functions to perform parameter estimations and visualse the results. 3) functions to optimize future policies using model predictive control (MPC). The stochastic model class from the original SEIRSplus package will be modified ASAP with the exact same extensions. Also, scenario specific functions will be added over the course of next week. 

#### Creating a SEIRSAgeModel object
We start our demo with the initialisation of our 'toolbox', as shown in the cell below. This is done by calling the SEIRSAgeModel class and defining all clinical and testing parameters, the initial condition and if we want to enable monte-carlo simulations. By default, if the argument monteCarlo and n_samples are ommited from the object initialisation, monte-carlo sampling of the incubation period is switched off by default. **It is detrimental that Nc and all initial conditions (denoted 'initX') are numpy arrays. The initial conditions must be 1D arrays with the same size as the number of age categories considered in the metapopulation model. Nc must be a square 2D array with the same size as the number of age categories considered in the metapopulation model. For now, we ommit age-structuring as this is demonstrated later on.** We conveniently name our object 'model'.

In [ ]:
model = models.SEIRSAgeModel(initN   = np.array([11.43e6]), #must be a numpy array
                                     beta    = 0.032, 
                                     sigma   = 5.2,
                                     Nc = np.array([11.2]), #must be a numpy array
                                     sm = 0.50,
                                     m = (1-0.50)*0.81,
                                     h = (1-0.50)*0.15,
                                     c = (1-0.50)*0.04,
                                     dsm = 14,
                                     dm = 14,
                                     dhospital = 1,
                                     dh = 21,
                                     dcf = 18.5,
                                     dcr = 22.0,
                                     mc0 = 0.49,
                                     ICU = 2000,
                                     totalTests = 0,
                                     psi_FP = 0,
                                     psi_PP = 1,
                                     dq = 14,
                                     initE = np.array([1]), #must be a numpy array
                                     initSM = np.zeros(1), 
                                     initM = np.zeros(1),
                                     initH = np.zeros(1),
                                     initC = np.zeros(1),
                                     initHH = np.zeros(1),
                                     initCH = np.zeros(1),
                                     initR = np.zeros(1),
                                     initF = np.zeros(1),
                                     initSQ = np.zeros(1),
                                     initEQ = np.zeros(1),
                                     initSMQ = np.zeros(1),
                                     initMQ = np.zeros(1),
                                     initRQ = np.zeros(1),
                                     monteCarlo = False,
                                     n_samples = 100
                            )

#### Changing an object variable after intialisation

After initialising our 'model' it is still possible to change variables using the following syntax.

In [ ]:
model.beta = 0.032

#### Running your first simulation

A simulation is run by using the attribute function *sim*, which uses one argument, the simulation time T, as its input. 

In [ ]:
y = model.sim(40)

For advanced users: the numerical results of the simulation can be accessed directly be calling *object.X* or *object.sumX* where X is the name of the desired population pool. Both are numpy arrays. *Ojbect.X* is a 3D array of the following dimensions:
- x-dimension: number of age categories,
- y-dimesion: tN: total number of timesteps taken (one per day),
- z-dimension: n_samples: total number of monte-carlo simulations performed.

Object.sumX is a 2D array containing only the results summed over all age categorie and has the following dimensions,
- x-dimesion: tN: total number of timesteps taken (one per day),
- y-dimension: n_samples: total number of monte-carlo simulations performed.


#### Visualising the results

To quickly visualise simulation results, two attribute functions were created. The first function, *plotPopulationStatus*, visualises the number of susceptible, exposed, infected and recovered individuals in the population. The second function, *plotInfected*, by default visualises the number of heavily and critically infected individuals. Both functions require no user input to work but both have some optional arguments,

> plotPopulationStatus(filename),
> - filename: string with a filename + extension to save the figure. The figure is not saved per default.

> plotInfected(asymptotic, mild, filename),
> - asymptotic: set to *True* to include the supermild pool in the visualisation.
> - mild: set to *True* to include the mild pool in the visualisation.
> - filename: string with a filename + extension to save the figure. The figure is not saved per default.

In [ ]:
model.plotPopulationStatus()
model.plotInfected()

#### The use of checkpoints to change parameters on the fly

A cool feature of the original SEIRSplus package by Ryan McGee was the use of so-called *checkpoints* dictionary to change simulation parameters on the fly. In our modification, this feature is preserved. Below you can find an example of a *checkpoints* dictionary. The simulation will be started with the previously initialised parameters. After 40 days, social interaction will be limited by changing $N_c$ to 0.50 contacts per day. After 80 days, social restrictions are lifted and beta once more assumes its *business-as-usual* value. *checkpoints* is the only optional argument of the *sim* functions and is set to *None* per default.

In [ ]:
# Create checkpoints dictionary
chk = {'t':       [40,80], 
       'Nc':      [np.array([0.5]),np.array([11.2])]
      }
# Run simulation
y = model.sim(90,checkpoints=chk)
# Visualise
model.plotPopulationStatus()
model.plotInfected()

### Meta-population simulations

#### Creating an age-structured SEIRSAgeModel object
A first important challenge when using a deterministic model is to link the discrete levels of the control handle Nc (number of contacts) to specific government policies. A model extension that could be used to facilitate this is age-structuring. In this approach, all population pools are split in age-bins and the interactions between the age-bins are governed by a so-called interaction matrix. This modeling approach was recently used by a team of the London School of Hygiene and details can be found here [1]. Our model is written in such a way that incorporating age-structuring is only a matter of change the initial conditions and Nc-matrix sizes. We can already run simulations using the Belgian interaction matrix shown below and we will run the controller using this model in the very near future.

We now initialise a second, age-structured, 'toolbox', as shown in the cell below.  This is done in exactly the same way as before but instead of size-one numpy arrays we will now use a 16x16 interaction interaction matrix and 16x0 size initial conditions, as was demonstrated by the London School of Hygiene. Interaction matrices are publicly available here [2]. We conveniently name our object 'ageModel'.

[1] https://www.thelancet.com/journals/lanpub/article/PIIS2468-2667(20)30073-6/fulltext 

[2] https://github.com/kieshaprem/covid19-agestructureSEIR-wuhan-social-distancing/tree/master/data

<img src="../figs/BELinteractPlot.png"
     alt="interact"
     height="1000" width="1000"     
     style="float: left; margin-right: 500px;" /> 

In [ ]:
# Load interaction matrices
Nc_home = np.loadtxt("../data/Interaction_matrices/Belgium/BELhome.txt", dtype='f', delimiter='\t')
Nc_work = np.loadtxt("../data/Interaction_matrices/Belgium/BELwork.txt", dtype='f', delimiter='\t')
Nc_schools = np.loadtxt("../data/Interaction_matrices/Belgium/BELschools.txt", dtype='f', delimiter='\t')
Nc_others = np.loadtxt("../data/Interaction_matrices/Belgium/BELothers.txt", dtype='f', delimiter='\t')
Nc_all = np.loadtxt("../data/Interaction_matrices/Belgium/BELall.txt", dtype='f', delimiter='\t')
initN = np.loadtxt("../data/Interaction_matrices/Belgium/BELagedist.txt", dtype='f', delimiter='\t')

In [ ]:
ageModel = models.SEIRSAgeModel(initN   = initN,         #16x0 numpy array
                                     beta    = 0.032, 
                                     sigma   = 5.2,
                                     Nc = Nc_all,        #16x16 numpy array
                                     sm = 0.50,
                                     m = (1-0.50)*0.81,
                                     h = (1-0.50)*0.15,
                                     c = (1-0.50)*0.04,
                                     dsm = 14,
                                     dm = 14,
                                     dhospital = 1,
                                     dh = 21,
                                     dcf = 18.5,
                                     dcr = 22.0,
                                     mc0 = 0.49,
                                     ICU = 2000,
                                     totalTests = 0,
                                     psi_FP = 0,
                                     psi_PP = 1,
                                     dq = 14,
                                     initE = np.ones(16),
                                     initSM = np.zeros(16),
                                     initM = np.zeros(16),
                                     initH = np.zeros(16),
                                     initC = np.zeros(16),
                                     initHH = np.zeros(16),
                                     initCH = np.zeros(16),
                                     initR = np.zeros(16),
                                     initF = np.zeros(16),
                                     initSQ = np.zeros(16),
                                     initEQ = np.zeros(16),
                                     initSMQ = np.zeros(16),
                                     initMQ = np.zeros(16),
                                     initRQ = np.zeros(16),
                                     monteCarlo = False
                            )

In [ ]:
# Create checkpoints dictionary
chk = {'t':       [40,80], 
       'Nc':      [Nc_home,Nc_all]
      }
# Run simulation
y = ageModel.sim(90,checkpoints=chk)
# Visualise
model.plotPopulationStatus()
model.plotInfected()

## Calibrating $\beta$  in a *business-as-usual* scenario ($N_c = 11.2$)

### Performing a least-squares fit

The 'SEIRSAgeModel' class contains a function to fit the model to selected data (*fit*) and one function to visualise the result (*plotFit*). Our code uses the **genetic algorithm** from scipy to perform the optimisation. The *fit* function has the following basic syntax,

> sim(data, parNames, positions, bounds, weights)
> - data: a list containing the dataseries (dtype=np array) to fit the model to. For now, dataseries must be of equal length.
> - parNames: a list containing the names (dtype=string) of the model parameters to be fitted.
> - positions: a list containing the model states (dtype=np.array) used to calculate the sum of least squares. Each dataseries must be matched to a certain (sum of) model state(s). If multiple entries are provided these are added togheter. The order of the states is given according to the following vector, where S has index 0: (S, E, SM, M, H, C, HH, CH, R, F, SQ, EQ, SMQ, MQ, RQ).


The following arguments are optional,
> - checkpoints: checkpoint dictionary can be used to calibrate under specific scenarios such as policy changes (default: None).
> - setvar: True to replace fitted values in model object after fit is performed (default: False).
> - disp: Show sum-of-least-squares after each optimisation iteration (default: True).
> - polish: True to use a Nelder–Mead simplex to polish the final result (default: True).
> - maxiter: Maximum number of iterations (default: 30).
> - popsize: Population size of genetic algorithm (default: 10).

The genetic algorithm will by default use all cores available for the optimisation. Using the *fit* attribute, it is possible to calibrate any number of model parameters to any sets of data. We do note that fitting the parameters sm,m,h and c requires modification of the source code. In the example below, the transmission parameter $\beta$ is sought after using two dataseries. The first is the number of patients in need of intensive care and the second is the total number of people in the hospital. The number of patients in ICU is matched with the CH population pool while the number of hospitalisations is matched with the sum of the HH and CH population pools.

In [ ]:
# hardcoded data vectors
ICUvect= np.array([[5,24,33,53,79,100,130,164,238,290,322,381,474,605,690,789,867,927,1021,1088,1144,1205,1245,1261,1257,1260,1276,1285,1278,1262,1232,1234,1223]])
hospital = np.array([[58,97,163,264,368,496,648,841,1096,1380,1643,1881,2137,2715,3068,3640,4077,4468,4884,4975,5206,5358,5492,5509,5600,5738,5692,5590,5610,5635,5409,5393,5536]])
# vector with dates
index=pd.date_range('2020-03-13', freq='D', periods=ICUvect.size)
# data series used to calibrate model must be given to function 'plotFit' as a list
idx = -26
index = index[0:idx]
data=[np.transpose(ICUvect[:,0:idx]),np.transpose(hospital[:,0:idx])]
# set optimisation settings
parNames = ['beta'] # must be a list!
positions = [np.array([7]),np.array([6,7])] # must be a list!
bounds=[(10,60),(0.01,0.035)] # must be a list!
weights = np.array([1,1])
# run optimisation
theta = model.fit(data,parNames,positions,bounds,weights,setvar=True,maxiter=100)

### Visualising the fit

Visualising the resulting fit is easy and can be done using the plotFit function. The functions uses the following basic syntax,

> plotFit(index,data,positions)
> - index: vector with timestamps corresponding to data.
> - data: list containing the dataseries (dtype=np array) to fit the model to. For now, dataseries must be of equal length.
> - positions: list containing the model states (dtype=np.array) used to calculate the sum of least squares.

The following arguments are optional,
> - dataMkr: list containing the markers (dtype=str) to be used to visualise the data. Default value works up to five dataseries and is equal to: ['o','v','s','*','^'].
>- modelClr: list containing the colors (dtype=str) to be used to visualise the model fit. Default value works up to five dateseries and is equal to: ['green','orange','red','black','blue'].
> - legendText: tuple containing the legend entries. Disabled per default.
> - titleText: string containing the fit title. Disable per default.
> - filename: string with a filename + extension to save the figure. The figure is not saved per default.

In [ ]:
# plot result
model.plotFit(index,data,positions,modelClr=['red','orange'],legendText=('ICU (model)','ICU (data)','Hospital (model)','Hospital (data)'),titleText='Belgium')

## Model Predictive control (MPC)

### Optimising government policy

#### Process control for the layman

As we have the impression that the control part, which we see as our main addition to the problem, is more difficult to grasp for the layman, here is a short intro to process control. Experts in control are welcome to skip this section.

A predictive model consists of a set of equations and aims to predict how the system will behave in the future given a certain input. Process control flips this around and aims at determining what input is needed to achieve a desired system behavior (= goal). It is a tool that helps us in “controlling” how we want a system to behave. It is commonly applied in many industries, but also in our homes (e.g. central heating, washing machine). It's basically everywhere. Here's how it works. An algorithm monitors the deviation between the goal and the true system value and then computes the necessary action to "drive" the system to its goal by means of an actuator (in industry this is typically a pump or a valve). Applying this to Covid-19, the government wants to "control" the spread of the virus in the population by imposing measures (necessary control actions) on the public (which is the actuator here) and achieve the goal that the number of severely sick people does not become larger than can be handled by the health care system. However, the way the population behaves is a lot more complex compared to the heating control in our homes since not only epidemiology (virus spread) but also different aspects of human behavior on both the individual and the societal level (sociology, psychology, economy) are involved. This leads to multiple criteria we want to ideally control simultaneously and we want to use the "smartest" algorithm we can get our hands on.

#### The optimizePolicy function

The 'SEIRSAgeModel' class contains an implementation of the MPC controller in the function *optimizePolicy*. For now, the controller minimises a weighted squared sum-of-errors between multiple setpoints and model predictions. The algorithm can use any variable to control the virus outbreak, but we recommend sticking with the number of random daily contacts $N_c$ and the total number of random tests ('totalTests') as only these have been tested. We also recommend disabling age-structuring in the model before running the MPC as this feature requires discretisation of the interaction matrix to work which is not yet implemented. Future work will extend the MPC controller to work with age-structuring feature inherent to the model. Future work is also aimed at including an economic cost function to discriminate between control handles. Our MPC uses the **genetic algorithm** from scipy.optimize to perform the optimisation, we recommend using at least a populationsize of 20 and at least 100 iterations to ensure that the trajectory is 'optimal'. The *optimizePolicy* function has the following basic syntax,

> optimizePolicy(parNames, bounds, setpoints, positions, weights)
> - parNames: a list containing the names (dtype=string) of the model parameters to be used as a control handle.
> - bounds: A list containing the lower- and upper boundaries of each parameter to be used as a control handle. Each entry in the list should be a 1D numpy array containing the lower- and upper bound for the respective control handle.
> - setpoints: A list with the numerical values of the desired model output.
> - positions: a list containing the model states (dtype=np.array) used to calculate the sum of least squares. Each modelouput in the given position is matched with a provided setpoint. If multiple position entries are provided, the output in these positions is added togheter. The order of the states is given according to the following vector, where S has index 0: (S, E, SM, M, H, C, HH, CH, R, F, SQ, EQ, SMQ, MQ, RQ).


The following arguments are optional,
> - policy_period: length of one policy interval (default: 7 days).
> - N: number of future policy intervals to be optimised, also called 'control horizon' (default: 6).
> - P: number of policy intervals over which the sum of squared errors is calculated, also called 'prediction horizon' (default:12).
> - disp: Show sum-of-least-squares after each optimisation iteration (default: True).
> - polish: True to use a Nelder–Mead simplex to polish the final result (default: True).
> - maxiter: Maximum number of iterations (default: 100).
> - popsize: Population size of genetic algorithm (default: 20).

The function returns a one-dimensional list containing the optimal values of the control handles. The length of this list is equal to the length of the control horizon (N) times the number of control handles. The list thus lists all control handles and their optimal values in their respective order. **The optimal policy is assigned to the SEIRSAgeModel object and is only overwritten when a new optimisation is performed. Future work could include the creation of a new object for every optimal policy.** The genetic algorithm will by default use all cores available for the optimisation.

In [ ]:
parNames = ['Nc','totalTests']
bounds = [np.array([0,11.2]),np.array([0,1e6])]
setpoints = [1200,5000]
positions = [np.array([7]),np.array([6,7])]
weights = [1,0]
model.optimizePolicy(parNames,bounds,setpoints,positions,weights,policy_period=30,N=6,P=12,polish=False,maxiter=1,popsize=10)

### Visualising the effect of government policy

Visualising the resulting optimal policy is easy and can be done using the plotOptimalPolicy function. We note that the functionality of*plotOptimalPolicy** is for now, very basic and will be extended in the future. The function is heavily based on the *plotInfected* visualisation. The function uses the following basic syntax,

> plotOptimalPolicy(parNames,setpoints,policy_period)
> - parNames: a list containing the names (dtype=string) of the model parameters to be used as a control handle.
> - setpoints: A list with the numerical values of the desired model output.
> - policy_period: length of one policy interval (default: 7 days).

The following arguments are optional,
> - asymptotic: set to *True* to include the supermild pool in the visualisation.
> - mild: set to *True* to include the mild pool in the visualisation.
> - filename: string with a filename + extension to save the figure. The figure is not saved per default.

In [ ]:
model.plotOptimalPolicy(parNames,setpoints,policy_period=14)

## Scenario-specific extensions

### A function which uses the past policies to quickly recreate the Belgian ICU and hospitalisation curves up-to-date

This function will be used to quickly propose MPC optimal policies and to perform scenario analysis about the future.

# To be continued..